In [7]:
# !pip install -q -r ../requirements.txt

In [1]:
import sys
sys.path.append("..")

from file import read_df

raw_df = read_df("/mnt/data/data/Data_Ner/Data_Ner.xlsx")
raw_df.head()

,id,title,label
0,91227.jpg,TIỆM TÓC Ngọc Bích CẮT - UỐN - DUỖI - NHUỘM CẠ...,K K N N K K K K K K K K K K K K K K K K K K K ...
1,94739.jpg,CẮT TÓC NAM - NỮ Linh CẮT DUỖI NHUỘM UỐN KỸ TH...,K K K K K N K K K K K K K K K K K K K A A A A ...
2,91831.jpg,Tiệm Tóc MỸ LINH UỐN DUỖI NHUỘM LÀM MÓNG GEL N...,K K N N K K K K K K K K K K K K O O O O O
3,94567.jpg,"Hair Salon Thủy * CẮT, UỐN, GỘI * DUỖI, NHUỘM,...",K K N K K K K K K K K K K K K
4,94207.jpg,Q QUEEN JAPAN COSMETICS - Tuyển CTV Mỹ phẩm ch...,N N K K K K K K K K K K K K K K K K K K K K K ...


In [2]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2729 entries, 0 to 2728
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2729 non-null   object
 1   title   2729 non-null   object
 2   label   2729 non-null   object
dtypes: object(3)
memory usage: 64.1+ KB


In [6]:
from dataset import *
from fine_tune.constants import *

poi_dataset = get_conll_dataset(clean_data(raw_df), labels=LABELS)
poi_dataset 

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 2694
})

In [7]:
sample_dataset = poi_dataset.select(range(10))
sample_dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 10
})

In [8]:
from adept_augmentations import EntitySwapAugmenter

augmenter = EntitySwapAugmenter(poi_dataset)
augmented_dataset = augmenter.augment(N=4)
augmented_dataset

Map:   0%|          | 0/2694 [00:00<?, ? examples/s]

Map: 100%|██████████| 2694/2694 [00:00<00:00, 4201.65 examples/s]


Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 10739
})

In [11]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
train_dataset = get_train_dataset(augmented_dataset, tokenizer)
train_dataset

/mnt/data/src/ner/.venvaug/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 10739/10739 [00:01<00:00, 8224.63 examples/s]


Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 10739
})

In [13]:
import json

with open("train_sample.json", "w") as f:
    json.dump(train_dataset[1], f)